# Task 8.2 Open QASM3

## Objective 1 : Interoperate different versions of QASM3 with qiskit


### Exporting to QASM 3



* use ```dump()``` to export to a file
* use ```dumps()``` to export to a Open QASM3 string

#### Exporter

* Exporter main class
* includes --> stdgates.inc
* basis_gate --> gates defined in the backend
* disable_constants
* alias_classic_registers
* allow_aliasing
* indent
* experimental
* annotation_handlers

#### dump

* export circuit to stream

#### dumps

* export circuit to string

#### QASM3ExporterError

* raise error during running QASM3 Exporter

#### ExeperimentalFeatures



### Importing from QASM 3



* use ```load``` to import from file
* use ```loads``` to import from stream

#### QASM3ImoporterError

#### ExperimentalImportInterface

* in Rust
* load_experimental
* loads_experimental